In [1]:
file_vv30K = open('data/vv30K.index.txt', encoding='UTF-8')
lines_vv30K = file_vv30K.readlines()
file_vv30K.close()

In [2]:
specials = ['gi','yêm','yên','yêu']
starts =  ['','b','c','ch','d','g','gh','gi','h','k','kh','l','m','n','ng','ngh','nh','ph','q','r','s','t','th','tr','v','x','đ']
ends = ['a', 'ac', 'ach', 'ai', 'am', 'an', 'ang', 'anh', 'ao', 'ap', 'at', 'au', 'ay', 'e', 'em', 'en', 'eng', 'eo', 'ep', 'et', 'i', 'ia', 'ich', 'im', 'in', 'inh', 'ip', 'it', 'iu', 'iêc', 'iêm', 'iên', 'iêng', 'iêp', 'iêt', 'iêu', 'o', 'oa', 'oach', 'oai', 'oan', 'oang', 'oanh', 'oat', 'oay', 'oc', 'oe', 'oi', 'om', 'on', 'ong', 'oong', 'op', 'ot', 'oăc', 'oăn', 'oăng', 'oăt', 'u', 'ua', 'uc', 'ui', 'um', 'un', 'ung', 'up', 'ut', 'uy', 'uya', 'uych', 'uynh', 'uyt', 'uyên', 'uyêt', 'uân', 'uâng', 'uât', 'uê', 'uôc', 'uôi', 'uôm', 'uôn', 'uông', 'uôt', 'uơ', 'y', 'âc', 'âm', 'ân', 'âng', 'âp', 'ât', 'âu', 'ây', 'ê', 'êch', 'êm', 'ên', 'ênh', 'êp', 'êt', 'êu', 'ô', 'ôc', 'ôi', 'ôm', 'ôn', 'ông', 'ôp', 'ôt', 'ăc', 'ăm', 'ăn', 'ăng', 'ăp', 'ăt', 'ơ', 'ơi', 'ơm', 'ơn', 'ơp', 'ơt', 'ư', 'ưa', 'ưc', 'ưi', 'ưng', 'ưt', 'ưu', 'ươc', 'ươi', 'ươm', 'ươn', 'ương', 'ươp', 'ươt', 'ươu']
tones = ['aáàảãạ','âấầẩẫậ','ăắằẳẵặ','eéèẻẽẹ','êếềểễệ','iíìỉĩị','oóòỏõọ','ôốồổỗộ','ơớờởỡợ','uúùủũụ','ưứừửữự','yýỳỷỹỵ']

specials.sort()
starts.sort()
ends.sort()
tones.sort()

file_encoder = open('output/encoder.txt', 'w', encoding='UTF-8')
file_encoder.write(str(len(specials)))
file_encoder.write(' ')
file_encoder.write(str(len(starts)))
file_encoder.write(' ')
file_encoder.write(str(len(ends)))
file_encoder.write(' ')
file_encoder.write(str(len(tones)))
file_encoder.write('\n')
for special in specials:
    file_encoder.write(special)
    if (special == specials[-1]):
        file_encoder.write('\n')
    else:
        file_encoder.write(',')
for start in starts:
    file_encoder.write(start)
    if (start == starts[-1]):
        file_encoder.write('\n')
    else:
        file_encoder.write(',')
for end in ends:
    file_encoder.write(end)
    if (end == ends[-1]):
        file_encoder.write('\n')
    else:
        file_encoder.write(',')
for tone in tones:
    file_encoder.write(tone)
    if (tone == tones[-1]):
        file_encoder.write('\n')
    else:
        file_encoder.write(',')
file_encoder.close()

In [3]:
def removeTone(word):
    index_tone = 0
    for i in reversed(range(len(word))):
        for j in tones:
            for k in range(1,len(j)):
                if word[i] == j[k]:
                    word = word[:i] + j[0] + word[i+1:]
                    index_tone = k;
                    return index_tone, word;
    return index_tone, word;

def encoderToInt(word):
    index_start = 0
    index_end = -1
    index_tone = 0
    
    word = word.lower()
    
    index_tone, word = removeTone(word)
    
    for i in range(len(specials)):
        if (word == specials[i]):
            index_start = 31
            index_end = i
            return (index_end << 8) + (index_start << 3) + index_tone;

    for i in reversed(range(len(starts))):
        if word.startswith(starts[i]):
            index_start = i
            break
            
    end = word[len(starts[index_start]):]
    if end in ends:
        index_end = ends.index(end)

    if index_end == -1:
        return -1
    return (index_end << 8) + (index_start << 3) + index_tone

def funcSort(word):
    vs = word.split(' ')
    return (encoderToInt(vs[0]) << 16) + encoderToInt(vs[1])

In [4]:
words = []
for line in lines_vv30K:
    line = line.split('\t')[0].lower()
    vs = line.split(' ')
        
    if len(vs) == 2 and encoderToInt(vs[0]) != -1 and encoderToInt(vs[1]) != -1:
        words.append(line)
    
words = list(set(words))
words.sort(key=funcSort)

In [5]:
file_dictionary = open('output/dictionary.txt', 'w', encoding='UTF-8')
file_dictionary.write(str(len(words)))
file_dictionary.write('\n')
for word in words:
    file_dictionary.write(word)
    if (word == words[-1]):
        file_dictionary.write('\n')
    else:
        file_dictionary.write(',')
        
file_dictionary.close()

In [6]:
import operator

start_words = {}
end_words = {}
for i in range(len(words)):
    vs = words[i].split(' ')
    if start_words.get(vs[0]) != None:
        start_words[vs[0]] = start_words[vs[0]] + 1;
    else:
        start_words[vs[0]] = 1

    if end_words.get(vs[1]) != None:
        end_words[vs[1]] = end_words[vs[1]] + 1;
    else:
        end_words[vs[1]] = 1

In [7]:
file_start_words = open('output/start_word.txt', 'w', encoding='UTF-8')
list_start_words = list(start_words.keys())
file_start_words.write(str(len(list_start_words)))
file_start_words.write('\n')
for word in list_start_words:
    file_start_words.write(word)
    if (word == list_start_words[-1]):
        file_start_words.write('\n')
    else:
        file_start_words.write(',')
file_start_words.close()

In [8]:
# Report
print(f'len(words): {len(words)}')
print(f'range encode: {encoderToInt("a")} to {encoderToInt("đượu")}')
print(f'len(start_words): {len(start_words)}')
print(f'len(end_words): {len(end_words)}')

len(words): 21403
range encode: 0 to 35029
len(start_words): 3767
len(end_words): 4287
